## Notebook Setup

In [1]:
config_file = "notebook_parameters_default.yml"

In [2]:
import os
import requests
import shutil
import sys
import warnings
from pathlib import Path
from tqdm import tqdm

import geopandas as gpd
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    import gplately
import joblib
import numpy as np
import pandas as pd
import pygplates
import rioxarray
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    from gplately import PlotTopologies

from lib.check_files import check_plate_model
from lib.extract_data import convert_age_to_depth
from lib.extract_data.create_lip_conjugates import create_lip_conjugates
from lib.extract_data.crustal_co2 import calculate_crustal_co2
from lib.extract_data.crustal_thickness import calculate_crustal_thickness
from lib.extract_data.lip_reconstruction import (
    calculate_depth_contours,
    mp_wrapper_for_shapefile_to_raster,
    rotation_LIP_shapefile_and_buffer,
)
from lib.extract_data.paleobathymetry import (
    # add_Pacific_synthetic_seamounts,
    calculate_paleobathymetry,
)
from lib.extract_data.paleotopography import paleotopography_job
from lib.extract_data.paleotopography.create_present_day_features import create_present_day_features
from lib.extract_data.paleotopography.tween_paleoshorelines_inplace import tween_paleoshorelines_inplace
from lib.load_params import get_params
from lib.plate_models import get_plate_reconstruction, get_plot_topologies

# Sediment thickness modules
sedthick_path = os.path.join(
    "submodules",
    "predicting-sediment-thickness",
)
if sedthick_path not in sys.path:
    sys.path.insert(0, sedthick_path)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    from ocean_basin_proximity import (
        generate_and_write_proximity_data_parallel,
        generate_input_points_grid as generate_input_points_grid_seds,
    )
    from predict_sediment_thickness import (
        predict_sedimentation,
        write_grd_file,
    )

# Carbonate thickness modules
carbonate_path = os.path.join(
    "submodules",
    "CarbonateSedimentThickness",
)
if carbonate_path not in sys.path:
    sys.path.insert(0, carbonate_path)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    from carbonate_sediment_thickness import (
        calc_sedimentation,
        generate_input_points_grid as generate_input_points_grid_carb,
        write_data,
    )

In [3]:
params = get_params(config_file, notebook="00a")

# Directory for output
output_dir = params["extracted_data_dir"]

# Overwrite previous outputs
overwrite = params["overwrite_output"]
overwrite = False

# Plate model
plate_model_name = params["plate_model"]["plate_model_name"]
use_provided_plate_model = params["plate_model"]["use_provided_plate_model"]

# Timespan for analysis
min_time = params["timespan"]["min"]
max_time = params["timespan"]["max"]

# Number of processes to use
n_jobs = params["n_jobs"]

# Cleanup working directories
# cleanup = params["cleanup"]
cleanup = False

times = range(min_time, max_time + 1)

In [4]:
Path(output_dir).mkdir(parents=True, exist_ok=True)

plate_model_dir = "plate_model"
if use_provided_plate_model:
    check_plate_model(plate_model_dir, verbose=True)
    plate_model_name = None
plate_model = get_plate_reconstruction(
    model_name=plate_model_name,
    model_dir=plate_model_dir,
)

if use_provided_plate_model:
    coastlines_filenames = [os.path.join(
        plate_model_dir,
        "StaticGeometries",
        "AgeGridInput",
        "CombinedTerranes.gpml",
    )]
    gplot = PlotTopologies(
        plate_model,
        coastlines=coastlines_filenames,
        continents=coastlines_filenames,
    )
else:
    gplot = get_plot_topologies(
        model_name=plate_model_name,
        model_dir=plate_model_dir,
        plate_reconstruction=plate_model,
    )

## Seafloor age

In [5]:
seafloor_age_output_dir = os.path.join(output_dir, "SeafloorAge")
spreading_rate_output_dir = os.path.join(output_dir, "SpreadingRate")

run_seafloor_age = overwrite
if not run_seafloor_age:
    for time in times:
        p = Path(seafloor_age_output_dir, f"seafloor_age_{time:0.0f}Ma.nc")
        q = Path(spreading_rate_output_dir, f"spreading_rate_{time:0.0f}Ma.nc")
        if not (p.exists() and q.exists()):
            run_seafloor_age = True
            break

if run_seafloor_age:
    seafloor_grid = gplately.SeafloorGrid(
        gplot.plate_reconstruction,
        gplot,
        min_time=min_time,
        max_time=max_time,
        ridge_time_step=1,
        save_directory=os.path.join(output_dir, "seafloor_age_output"),
    )
    seafloor_grid.reconstruct_by_topologies()
    for i in ["SEAFLOOR_AGE", "SPREADING_RATE"]:
        seafloor_grid.lat_lon_z_to_netCDF(
            i,
            nprocs=n_jobs,
        )

    for which in "SEAFLOOR_AGE", "SPREADING_RATE":
        d_old = os.path.join(output_dir, "seafloor_age_output", which)
        d_new = os.path.join(output_dir, "".join([i.capitalize() for i in which.split("_")]))
        os.makedirs(d_new, exist_ok=True)
        for t in times:
            src = f"{which}_grid_{t:0.2f}Ma.nc"
            dst = f"{which.lower()}_{t:0.0f}Ma.nc"
            os.rename(
                os.path.join(d_old, src),
                os.path.join(d_new, dst)
            )
        os.rmdir(d_old)
    shutil.rmtree(os.path.join(output_dir, "seafloor_age_output"))

## Sediment thickness

In [6]:
sedthick_output_dir = os.path.join(output_dir, "SedimentThickness")
os.makedirs(sedthick_output_dir, exist_ok=True)

run_sedthick = overwrite
if not run_sedthick:
    for time in times:
        p = Path(sedthick_output_dir, f"sediment_thickness_{time:0.0f}Ma.nc")
        if not p.exists():
            run_sedthick = True
            break

if run_sedthick:
    # Values taken from predicting-sediment-thickness scripts
    initial_grid_spacing = 1.0
    final_grid_spacing = 0.1
    max_mean_distance = 3000
    max_age = 191.87276
    mean_age =  61.18406823
    mean_distance = 1835.28118479
    variance_age = 1934.6999014
    variance_distance = 1207521.8995806
    age_distance_polynomial_coefficients = [
        5.441401190368497,  0.46893096, -0.07320928, -0.24077496, -0.10840657,
        0.00381672,  0.06831728,  0.01179914,  0.01158149, -0.39880562,
    ]
    input_points_initial = generate_input_points_grid_seds(initial_grid_spacing)[0]
    input_points_final = generate_input_points_grid_seds(final_grid_spacing)[0]

    seafloor_age_filenames = [
        (
            os.path.join(output_dir, "SeafloorAge", f"seafloor_age_{time:0.0f}Ma.nc"),
            time,
        )
        for time in times
    ]

    sedthick_workdir = os.path.join(output_dir, "sedthick_outputs")
    os.makedirs(sedthick_workdir, exist_ok=True)

    if use_provided_plate_model:
        sedthick_cobs = [
            os.path.join(plate_model_dir, "North_America_COBs.gpml"),
            os.path.join(
                plate_model_dir,
                "StaticGeometries",
                "AgeGridInput",
                "Global_EarthByte_GeeK07_COB_Terranes.gpml",
            ),
        ]
    else:
        sedthick_cobs = gplot._continents.filenames

    generate_and_write_proximity_data_parallel(
        input_points=input_points_initial,
        rotation_filenames=gplot.plate_reconstruction.rotation_model.filenames,
        proximity_filenames=sedthick_cobs,
        proximity_features_are_topological=False,
        proximity_feature_types=None,
        topological_reconstruction_filenames=gplot.plate_reconstruction.topology_features.filenames,
        age_grid_filenames_and_paleo_times=seafloor_age_filenames,
        time_increment=1,
        output_distance_with_time=True,
        output_mean_distance=True,
        output_standard_deviation_distance=True,
        output_directory=sedthick_workdir,
        max_topological_reconstruction_time=None,
        continent_obstacle_filenames=gplot._coastlines.filenames,
        anchor_plate_id=0,
        proximity_distance_threshold_radians=None,
        clamp_mean_proximity_distance_radians=max_mean_distance / gplately.EARTH_RADIUS,
        output_grd_files=(initial_grid_spacing, final_grid_spacing),
        num_cpus=n_jobs,
    )

    sedthick_output_template = os.path.join(
        sedthick_output_dir,
        r"sediment_thickness_{:0.0f}Ma.nc",
    )
    age_grid_filename_template = os.path.join(
        output_dir,
        "SeafloorAge",
        r"seafloor_age_{:0.0f}Ma.nc",
    )
    distance_grid_filename_template = os.path.join(
        sedthick_workdir,
        (
            f"mean_distance_{final_grid_spacing:0.1f}d"
            + r"_{:0.1f}.nc"
        ),
    )

    def func(
        output_filename,
        input_points,
        age_grid_filename,
        distance_grid_filename,
        mean_age,
        mean_distance,
        variance_age,
        variance_distance,
        age_distance_polynomial_coefficients,
        max_age,
        max_distance,
        grid_spacing,
    ):
        result = predict_sedimentation(
            input_points=input_points,
            age_grid_filename=age_grid_filename,
            distance_grid_filename=distance_grid_filename,
            mean_age=mean_age,
            mean_distance=mean_distance,
            variance_age=variance_age,
            variance_distance=variance_distance,
            age_distance_polynomial_coefficients=age_distance_polynomial_coefficients,
            max_age=max_age,
            max_distance=max_distance,
        )
        write_grd_file(
            output_filename,
            output_data=result,
            grid_spacing=grid_spacing,
            num_grid_longitudes=None,  # not used
            num_grid_latitudes=None,  # not used
        )

    with joblib.Parallel(n_jobs) as parallel:
        parallel(
            joblib.delayed(func)(
                output_filename=sedthick_output_template.format(time),
                input_points=input_points_final,
                age_grid_filename=age_grid_filename_template.format(time),
                distance_grid_filename=distance_grid_filename_template.format(time),
                mean_age=mean_age,
                mean_distance=mean_distance,
                variance_age=variance_age,
                variance_distance=variance_distance,
                age_distance_polynomial_coefficients=age_distance_polynomial_coefficients,
                max_age=max_age,
                max_distance=max_mean_distance,
                grid_spacing=final_grid_spacing,
            )
            for time in times
        )

## Carbonate thickness

In [7]:
carbonate_output_dir = os.path.join(output_dir, "CarbonateThickness")
os.makedirs(carbonate_output_dir, exist_ok=True)

carbonate_workdir = os.path.join(output_dir, "carbonate_outputs")
os.makedirs(carbonate_workdir, exist_ok=True)

### Paleobathymetry

In [8]:
run_paleobath = overwrite
if not run_paleobath:
    for time in times:
        p = Path(carbonate_workdir, f"paleobathymetry_{time:0.0f}Ma.nc")
        if not p.exists():
            run_paleobath = True
            break

if run_paleobath:
    # Basement depth
    def func(time, input_dir, output_dir):
        agegrid_filename = os.path.join(
            input_dir,
            f"seafloor_age_{time:0.0f}Ma.nc",
        )
        output_filename = os.path.join(
            output_dir,
            f"basement_depth_{time:0.0f}Ma.nc",
        )

        agegrid = gplately.Raster(agegrid_filename)
        basement_depth = agegrid.copy()
        basement_depth.data = convert_age_to_depth(agegrid.data)
        basement_depth.save_to_netcdf4(output_filename)


    with joblib.Parallel(n_jobs) as parallel:
        parallel(
            joblib.delayed(func)(
                time=time,
                input_dir=os.path.join(output_dir, "SeafloorAge"),
                output_dir=carbonate_workdir,
            )
            for time in times
        )

    # Large igneous provinces and seamounts
    paleobath_data_dir = os.path.join(
        "submodules",
        "paleobathymetry-workflow",
        "InputData",
    )
    gdf_seamounts = gpd.read_file(
        os.path.join(
            paleobath_data_dir,
            "Seamount_shapefile",
            "Johansson_etal_2018_VolcanicProvinces_v2_NW-edit.shp",
        )
    )
    gdf_LIPs = gpd.read_file(
        os.path.join(
            paleobath_data_dir,
            "LIP_shapefile",
            "LIPs_merged.shp",
        )
    )
    gdf_features = gpd.pd.concat([gdf_LIPs, gdf_seamounts])

    basement_depth = rioxarray.open_rasterio(
        os.path.join(carbonate_workdir, "basement_depth_0Ma.nc"),
        masked=True,
    )
    basement_depth = basement_depth.sel(band=1)
    basement_depth = basement_depth.drop(['band', 'spatial_ref'])
    basement_depth.rio.write_crs("epsg:4326", inplace=True)  # set crs

    os.makedirs(os.path.join(carbonate_workdir, "GDH1"), exist_ok=True)

    # LIPs
    calculate_depth_contours(
        basement_depth,
        gdf_features,
        contour_interval=50,
        LIP_depth_rounding=50,
        LIP_output_dir=carbonate_workdir,
        clip_contour_to_outline="yes",
        LIP_model_name="GDH1",
        path_contoured_LIPs=carbonate_workdir,
        feature_type="LIPs_seamounts",
        gdf_LIP_large=None,
    )

    # Conjugate LIPs
    create_lip_conjugates(
        LIP_contour_poly=os.path.join(
            carbonate_workdir,
            "contoured_LIPs_seamounts_0Ma.shp",
        ),
        rotation_model=plate_model.rotation_model.filenames,
        topology_features=plate_model.topology_features.filenames,
        LIP_conjugate_outdir=carbonate_workdir,
    )

    path_rotated_polygons = os.path.join(
        carbonate_workdir,
        "GDH1",
        "rotated_polygons",
    )
    os.makedirs(path_rotated_polygons, exist_ok=True)
    # Working on 0 Ma ONLY.
    # This is because we need to create swell_stats.txt (which is only created at 0 Ma)
    rotation_LIP_shapefile_and_buffer(
        time=0.0,
        path_0_shp=os.path.join(carbonate_workdir, "contoured_LIPs_seamounts_0Ma.shp"),
        rotation_filenames=plate_model.rotation_model.filenames,
        path_rotated_polygons=path_rotated_polygons,
        feature_type="LIPs_seamounts",
        include_conjugate_LIPs="yes",
        LIP_output_dir=carbonate_workdir,
        LIP_model_name="GDH1",
        cooling_model="GDH1",
        large_LIP_path=None,
        buffer_radius_deg=1.0,
        RHCW_age_depth_interp=None,
    )

    with joblib.Parallel(n_jobs) as parallel:
        parallel(
            joblib.delayed(rotation_LIP_shapefile_and_buffer)(
                time=time,
                path_0_shp=os.path.join(carbonate_workdir, "contoured_LIPs_seamounts_0Ma.shp"),
                rotation_filenames=plate_model.rotation_model.filenames,
                path_rotated_polygons=path_rotated_polygons,
                feature_type="LIPs_seamounts",
                include_conjugate_LIPs="yes",
                LIP_output_dir=carbonate_workdir,
                LIP_model_name="GDH1",
                cooling_model="GDH1",
                large_LIP_path=None,
                buffer_radius_deg=1.0,
                RHCW_age_depth_interp=None,
            )
            for time in times
        )
        parallel(
            joblib.delayed(mp_wrapper_for_shapefile_to_raster)(
                time=time,
                path_rotated_polygons=path_rotated_polygons,
                feature_type="LIPs_seamounts",
                cooling_model="GDH1",
                grid_spacing=0.1,
                lon_min=-180,
                lon_max=180,
                lat_min=-90,
                lat_max=90,
                path_output_grids=carbonate_workdir,
            )
            for time in times
        )

        # Create combined paleobathymetry
        parallel(
            joblib.delayed(calculate_paleobathymetry)(
                sedthick_filename=os.path.join(
                    output_dir,
                    "SedimentThickness",
                    f"sediment_thickness_{time:0.0f}Ma.nc",
                ),
                basement_depth_filename=os.path.join(
                    carbonate_workdir,
                    f"basement_depth_{time:0.0f}Ma.nc",
                ),
                lip_height_filename=os.path.join(
                    carbonate_workdir,
                    f"reconstructed_LIPs_seamounts_{time:0.0f}Ma.nc",
                ),
                output_filename=os.path.join(
                    carbonate_workdir,
                    f"paleobathymetry_{time:0.0f}Ma.nc",
                ),
            )
            for time in times
        )

### Carbonate thickness

In [9]:
run_carbonate = overwrite
if not run_carbonate:
    for time in times:
        p = Path(carbonate_output_dir, f"carbonate_thickness_{time:0.0f}Ma.nc")
        if not p.exists():
            run_carbonate = True
            break

if run_carbonate:
    ccd_curve_filename = os.path.join(
        carbonate_path,
        "input_data",
        "Boss_Wilkinson_1991_global_CCD.txt",
    )
    max_carbonate_decomp_sed_rate_cm_per_ky_curve_filename = os.path.join(
        carbonate_path,
        "input_data",
        "sed_rate_v6.txt",
    )

    # Create symlinks for seafloor age and bathymetry files
    # (necessary for carbonate thickness function)
    for time in times:
        # Seafloor age
        link_path = Path(
            output_dir,
            "SeafloorAge",
            f"seafloor_age_{time:0.0f}.nc",
        )
        target_path = Path(
            output_dir,
            "SeafloorAge",
            f"seafloor_age_{time:0.0f}Ma.nc",
        )
        if link_path.exists():
            link_path.unlink()
        link_path.symlink_to(target_path.relative_to(link_path.parent))

        # Bathymetry
        link_path = Path(
            carbonate_workdir,
            f"paleobathymetry_{time:0.0f}.nc",
        )
        target_path = Path(
            carbonate_workdir,
            f"paleobathymetry_{time:0.0f}Ma.nc",
        )
        if link_path.exists():
            link_path.unlink()
        link_path.symlink_to(target_path.relative_to(link_path.parent))

    grid_spacing = 0.5
    latitude_range = (-90, 90)
    longitude_range = (-180, 180)
    input_points = generate_input_points_grid_carb(
        grid_spacing_degrees=grid_spacing,
        latitude_range=latitude_range,
        longitude_range=longitude_range,
    )
    age_grid_filename_components = (
        os.path.join(output_dir, "SeafloorAge", "seafloor_age_"),  # prefix
        0,  # decimal places in time component
        "nc",  # file extension
    )
    bathymetry_filename_components = (
        os.path.join(carbonate_workdir, "paleobathymetry_"),  # prefix
        0,  # decimal places in time component
        "nc",  # file extension
    )


    def func(output_dir, grid_spacing, latitude_range, longitude_range, **kwargs):
        kwargs["rotation_model"] = pygplates.RotationModel(kwargs["rotation_model"])
        sediment_thickness_data = calc_sedimentation(**kwargs)
        (carbonate_decompacted_sediment_thickness_data,
        carbonate_compacted_sediment_thickness_data,
        carbonate_deposition_mask_data) = sediment_thickness_data
        time = kwargs.get("time")
        output_prefix = os.path.join(
            output_dir,
            f"carbonate_thickness_{time:0.0f}Ma",
        )
        write_data(
            data=carbonate_decompacted_sediment_thickness_data,
            output_filename_prefix=output_prefix,
            grid_spacing=grid_spacing,
            latitude_range=latitude_range,
            longitude_range=longitude_range,
        )


    with joblib.Parallel(n_jobs) as parallel:
        parallel(
            joblib.delayed(func)(
                output_dir=carbonate_output_dir,
                grid_spacing=grid_spacing,
                latitude_range=latitude_range,
                longitude_range=longitude_range,
                input_points=input_points,
                age_grid_filename_components=age_grid_filename_components,
                bathymetry_filename_components=bathymetry_filename_components,
                bathymetry_filename_oldest_time=max(times),
                topology_filenames=plate_model.topology_features.filenames,
                rotation_model=plate_model.rotation_model.filenames,
                ccd_curve_filename=ccd_curve_filename,
                max_carbonate_decomp_sed_rate_cm_per_ky_curve_filename=max_carbonate_decomp_sed_rate_cm_per_ky_curve_filename,
                carbonate_anchor_plate_id=0,
                time=time,
            )
            for time in times
        )

    # Clean up .xy files and symlinks for seafloor age and bathymetry files
    for time in times:
        # .xy carbonate thickness files
        xy_path = Path(
            carbonate_output_dir,
            f"carbonate_thickness_{time:0.0f}Ma.xy",
        )
        if xy_path.exists():
            xy_path.unlink()

        # Seafloor age
        link_path = Path(
            output_dir,
            "SeafloorAge",
            f"seafloor_age_{time:0.0f}.nc",
        )
        if link_path.exists() and link_path.is_symlink():
            link_path.unlink()

        # Bathymetry
        link_path = Path(
            carbonate_workdir,
            f"paleobathymetry_{time:0.0f}.nc",
        )
        if link_path.exists() and link_path.is_symlink():
            link_path.unlink()

## Crustal thickness

In [10]:
crust_output_dir = Path(
    output_dir,
    "CrustalThickness",
)
crust_output_dir.mkdir(parents=True, exist_ok=True)

run_crust = overwrite
if not run_crust:
    for time in times:
        p = Path(crust_output_dir, f"crustal_thickness_{time:0.0f}Ma.nc")
        if not p.exists():
            run_crust = True
            break

crust_workdir = Path(
    output_dir,
    "crust_outputs",
)
if run_crust:
    crust_workdir.mkdir(parents=True, exist_ok=True)

### Paleotopography

In [11]:
# Setup
area_threshold = 0.0
subdivision_depth = 2
resolution = 0.5  # degrees
ocean_elevation = -1000.0
shallow_elevation = -200.0
land_elevation = 200.0
mountain_relief = 3100.0
mountain_buffer_distance = 2.0  # degrees

run_paleotopo = overwrite
if not run_paleotopo:
    for time in times:
        p = Path(crust_workdir, f"paleotopo_{resolution:0.2f}d_{time:0.2f}Ma.nc")
        if not p.exists():
            run_paleotopo = True
            break

In [12]:
paleotopo_basename = "paleotopography-data"
paleotopo_dir = Path(crust_workdir, paleotopo_basename)
paleotopo_filename = Path(crust_workdir, paleotopo_basename + ".tgz")
paleotopo_url = "https://www.earthbyte.org/webdav/ftp/earthbyte/Paleotopography/paleotopography-data.tgz"
cookie_cut_dir = Path(crust_workdir, "cookie_cut")

if run_paleotopo:
    # Download paleotopography polygons
    if not paleotopo_filename.exists():
        with requests.get(paleotopo_url, stream=True) as r:
            r.raise_for_status()
            with paleotopo_filename.open("wb") as f:
                for chunk in tqdm(r.iter_content(chunk_size=1024000)):
                    f.write(chunk)
    # Extract archive
    shutil.unpack_archive(paleotopo_filename, extract_dir=paleotopo_dir)

    # Load polygons
    polygons_dir = Path(
        paleotopo_dir,
        "Paleogeography_Matthews2016_410-2Ma_Shapefiles",
    )
    gdf_paleotopo = []
    for d in polygons_dir.iterdir():
        if not d.is_dir():
            continue
        for fname in d.iterdir():
            if not fname.name.endswith(".shp"):
                continue
            which = fname.name.split("_")[0]
            gdf = gpd.read_file(fname)
            gdf["NAME"] = which
            gdf["ENV"] = which
            gdf_paleotopo.append(gdf)
    gdf_paleotopo = pd.concat(gdf_paleotopo, ignore_index=True).dropna(axis="columns")
    # gdf_paleotopo = gdf_paleotopo.drop(columns="PLATEID1").explode()

    # Cookie-cut polygons for plate model
    cookie_cut_dir.mkdir(exist_ok=True)
    sp_file = Path(crust_workdir, "static_polygons.shp")
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        pygplates.reconstruct(
            plate_model.static_polygons,
            plate_model.rotation_model,
            str(sp_file),
            0.0,
        )
    gdf_sp = gpd.read_file(sp_file).explode()
    gdf_sp = gdf_sp[gdf_sp.geometry.type == "Polygon"]
    gdf_sp = gdf_sp[["PLATEID1", gdf_sp.geometry.name]]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        gdf_paleotopo = gpd.overlay(
            gdf_paleotopo,
            gdf_sp,
            how="intersection",
        )
    inds = np.squeeze(np.where(
        (gdf_paleotopo["PLATEID1_2"].str.startswith("5"))
        | (gdf_paleotopo["PLATEID1_1"] == 616)
    ))
    for ind in inds:
        gdf_paleotopo.at[ind, "PLATEID1_2"] = gdf_paleotopo.at[ind, "PLATEID1_1"]
    gdf_paleotopo = gdf_paleotopo.rename(
        columns={"PLATEID1_2": "PLATEID1"}
    ).drop(columns="PLATEID1_1")

    # gdf_paleotopo = gpd.overlay(
    #     gdf_paleotopo,
    #     gdf_sp,
    #     how="intersection",
    #     keep_geom_type=True,
    # )
    for which in ("i", "sm", "m", "lm"):
        p = Path(cookie_cut_dir, f"{which}_402_2.shp")
        gdf_which = gdf_paleotopo[gdf_paleotopo["ENV"] == which]
        gdf_which.to_file(str(p))
else:
    gdf_paleotopo = []
    for which in ("i", "sm", "m", "lm"):
        p = Path(cookie_cut_dir, f"{which}_402_2.shp")
        gdf_paleotopo.append(gpd.read_file(p))
    gdf_paleotopo = pd.concat(gdf_paleotopo, ignore_index=True)

In [13]:
tween_dir = str(Path(crust_workdir, "tween_dir"))
Path(tween_dir).mkdir(exist_ok=True)
topography_filename = str(Path(paleotopo_dir, "topo15_3600x1800.nc"))
classes_filename = str(Path(crust_workdir, "present_day_topo_as_classes.nc"))
presentday_filename = str(Path(crust_workdir, "present_day_paleogeography.gmt"))

if run_paleotopo:
    create_present_day_features(
        topography_filename=topography_filename,
        classes_filename=classes_filename,
        features_filename=presentday_filename,
    )

In [14]:
# Takes an hour or two, so only run if necessary
tween_times = sorted(gdf_paleotopo["TIME"].unique())

run_tween = overwrite
if not run_tween:
    for t1, t2 in zip(tween_times[:-1], tween_times[1:]):
        for which in (
            "tweentest_land",
            "tweentest_ocean",
            "mountain_regression",
            "mountain_stable",
            "mountain_transgression",
        ):
            p = Path(tween_dir, f"{which}_{t1:0.2f}Ma_{t2:0.2f}Ma.gpmlz")
            if not p.exists():
                run_tween = True
                break

if run_tween:
    tween_paleoshorelines_inplace(
        tween_dir=tween_dir,
        basedir=str(cookie_cut_dir),
        rotation_filenames=plate_model.rotation_model.filenames,
        presentday_filename=presentday_filename,
        times=tween_times,
        resolution=resolution,
        nprocs=n_jobs,
        verbose=False,
    )

In [15]:
# Create paleotopography grids
if run_paleotopo:
    paleogeography_timeslice_list = sorted(gdf_paleotopo["TIME"].unique())
    paleogeography_timeslice_list.append(0.0)
    paleogeography_timeslice_list = np.array(paleogeography_timeslice_list)
    paleogeography_timeslice_list.sort()

    with joblib.Parallel(n_jobs) as parallel:
        parallel(
            joblib.delayed(paleotopography_job)(
                reconstruction_time=time,
                paleogeography_timeslice_list=paleogeography_timeslice_list,
                tween_basedir=tween_dir,
                reconstruction_basedir=str(Path(crust_workdir, "cookie_cut")),
                output_dir=crust_workdir,
                file_format="gpmlz",
                rotation_file=plate_model.rotation_model.filenames,
                COBterrane_file=gplot._continents.filenames,
                agegrid_file_template="",
                lowland_elevation=land_elevation,
                shallow_marine_elevation=shallow_elevation,
                max_mountain_elevation=mountain_relief,
                depth_for_unknown_ocean=ocean_elevation,
                sampling=resolution,
                mountain_buffer_distance_degrees=mountain_buffer_distance,
                area_threshold=area_threshold,
                grid_smoothing_wavelength_kms=None,
                merge_with_bathymetry=False,
                land_or_ocean_precedence='land',
                netcdf3_output=False,
                subdivision_depth=subdivision_depth,
                present_day_filename=presentday_filename,
            )
            for time in times
        )

### Crustal thickness

In [16]:
if run_crust:
    with joblib.Parallel(n_jobs) as parallel:
        parallel(
            joblib.delayed(calculate_crustal_thickness)(
                time=time,
                input_dir=str(crust_workdir),
                output_dir=str(crust_output_dir),
            )
            for time in times
        )

## Oceanic crustal $\mathrm{CO_2}$

In [17]:
co2_output_dir = Path(
    output_dir,
    "CrustalCO2",
)
co2_output_dir.mkdir(parents=True, exist_ok=True)

run_co2 = overwrite
if not run_co2:
    for time in times:
        p = Path(co2_output_dir, f"crustal_co2_{time:0.0f}Ma.nc")
        if not p.exists():
            run_co2 = True
            break

if run_co2:
    calculate_crustal_co2(
        times=times,
        seafloor_age_dir=seafloor_age_output_dir,
        output_dir=co2_output_dir,
        n_jobs=n_jobs,
    )

## Erosion

In [18]:
erodep_output_dir = Path(output_dir, "ErosionDeposition")
erodep_output_dir.mkdir(exist_ok=True)

run_erodep = False
erodep_times = range(0, 270, 5)
for time in erodep_times:
    if time <= min_time - 5 or time >= max_time + 5:
        continue
    p = Path(erodep_output_dir, f"erosion_deposition_{time:0.0f}Ma.nc")
    if not p.exists():
        run_erodep = True
        break

In [19]:
if run_erodep:
    erodep_workdir = Path(output_dir, "erodep_outputs")
    erodep_workdir.mkdir(exist_ok=True)

    erodep_url = "https://zenodo.org/records/14010839/files/erosion_deposition_files.zip?download=1"
    erodep_filename = Path(erodep_workdir, "erosion_deposition_files.zip")
    if not erodep_filename.exists():
        # Download archive from Zenodo
        with requests.get(erodep_url, stream=True) as r:
            r.raise_for_status()
            with erodep_filename.open("wb") as f:
                for chunk in tqdm(r.iter_content(chunk_size=1024000)):
                    f.write(chunk)
    # Extract archive
    shutil.unpack_archive(erodep_filename, extract_dir=erodep_workdir)

    # Copy missing files
    srcdir = Path(erodep_workdir, "erosion_deposition_files")
    for time in erodep_times:
        if time <= min_time - 5 or time >= max_time + 5:
            continue
        src = Path(srcdir, f"erodep{time:0.0f}Ma.nc")
        dst = Path(erodep_output_dir, f"erosion_deposition_{time:0.0f}Ma.nc")
        if dst.exists():
            continue
        shutil.copy2(src, dst)

## Cleanup

Remove all working directories to save space.

In [20]:
if cleanup:
    for d in (
        "carbonate_outputs",
        "crust_outputs",
        "erodep_outputs",
        "sedthick_outputs",
    ):
        p = Path(output_dir, d)
        if p.exists():
            shutil.rmtree(p)